In [1]:
import tensorflow as tf

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [3]:
import keras

Using TensorFlow backend.


In [4]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
from Abstract import Abstract
read_pre = '/data/liu/mimic3/USER_VECTORS/%s.csv'
aa = Abstract(read_prefix=read_pre, write_prefix=read_pre)
X_train_scaled = aa.read_data('lab_diag_train_scaled',None)
X_test_scaled = aa.read_data('lab_diag_test_scaled',None)

## Specify GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.layers import Input, Dense
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam

In [8]:
## Construct autoencoder model
model = Sequential()
model.add(Dense(5000,  activation='relu', input_shape=(13117,)))
model.add(Dense(128,  activation='linear', name="bottleneck"))
model.add(Dense(5000, activation='relu'))
model.add(Dense(13117,  activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5000)              65590000  
_________________________________________________________________
bottleneck (Dense)           (None, 128)               640128    
_________________________________________________________________
dense_2 (Dense)              (None, 5000)              645000    
_________________________________________________________________
dense_3 (Dense)              (None, 13117)             65598117  
Total params: 132,473,245
Trainable params: 132,473,245
Non-trainable params: 0
_________________________________________________________________


In [10]:
## Train model
encoder = Model(model.input, model.get_layer('bottleneck').output)
model.compile(loss='mse', optimizer = Adam())

model.fit(X_train_scaled,X_train_scaled, batch_size=5000, epochs=3, shuffle=True)


Epoch 1/3
37000/37000 [==============================] - 22s 596us/step - loss: 0.0784
Epoch 2/3
37000/37000 [==============================] - 21s 555us/step - loss: 0.0180
Epoch 3/3
37000/37000 [==============================] - 21s 563us/step - loss: 0.0184


In [11]:
representations = encoder.predict(np.concatenate((X_train_scaled, X_test_scaled), axis=0))  # bottleneck representation



In [12]:
representations.shape

(46250, 128)

In [14]:
representations[:2]

array([[-79.54719  , -51.86097  , -59.784878 , -34.812004 , -17.154598 ,
         57.76872  , -59.443817 ,  51.83057  ,  50.227425 ,  62.451515 ,
        -64.134285 , -44.729664 , -41.08687  , -27.206694 , -83.18298  ,
         43.851826 ,  72.819016 , -78.5613   ,  60.387184 ,  68.198265 ,
         62.175426 , -34.75865  ,  69.06553  ,  43.556763 , -55.076534 ,
         83.30189  ,  32.31986  ,  60.31306  ,  60.562336 ,  43.9537   ,
         80.002464 ,  62.458225 , -15.407141 , -30.135937 ,   8.513957 ,
        -73.47895  ,  57.139557 ,  62.26405  , -43.905163 ,  63.872627 ,
        -55.155872 ,  -0.7657843,  -9.014201 , -64.246605 ,  34.068794 ,
        -53.160583 ,  -6.3852863,  83.42692  ,  48.867424 ,  60.738735 ,
         43.074425 ,  54.66656  , -47.293873 ,  52.9292   ,  17.722717 ,
         -5.6023965,  47.75376  ,  47.776806 , -45.0529   ,  44.312042 ,
        -58.800545 ,  68.85031  ,  34.991154 ,  13.000304 ,  -7.6618285,
        -61.760605 ,  25.64993  ,  56.485313 , -49.

In [15]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [16]:
## Save 0 only as 0
def savetxt_compact(fname, x, fmt="%.6g", delimiter=','):
    with open(fname, 'w') as fh:
        for row in x:
            line = delimiter.join("0" if value == 0 else fmt % value for value in row)
            fh.write(line + '\n')
            
savetxt_compact(read_pre % 'lab_diag_128', representations, fmt='%.8f')

In [17]:
savetxt_compact(read_pre % 'lab_diag_128', representations, fmt='%.8f')


In [18]:
##
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
0.9046358852647245
GPU (s):
1.7261545551009476
GPU speedup over CPU: 0x
